### Word Embeddings

En esencia, las representaciones de palabras (word embeddings) son representaciones numéricas de datos del mundo real como texto, imágenes, voz o video. Se expresan como vectores de baja dimensionalidad, donde las distancias geométricas de dos vectores en el espacio vectorial son una proyección de las relaciones entre los dos objetos del mundo real que los vectores representan. En otras palabras, te ayudan a proporcionar una representación compacta de datos de diferentes tipos, mientras que simultáneamente te permiten comparar dos objetos de datos diferentes y decir cuán similares o diferentes son en una escala numérica.

Estas representaciones de baja dimensionalidad de datos del mundo real ayudan significativamente a un procesamiento y almacenamiento de datos a gran escala de manera eficiente, actuando como un medio de compresión con pérdida de los datos originales mientras se conservan sus propiedades importantes.

Los "word embeddings" se construyen con diferentes técnica y con modelos de aprendizaje automático, generalmente se pueden usar los word embeddings creados por ChatGPT (OpenAI), Google AI etc.





In [1]:
!pip install PyPDF2 langchain==0.3.9 langchain-community==0.3.9 langchain-core==0.3.21 langchain-experimental==0.3.3 huggingface-hub==0.23.2 instructor==1.7.0 InstructorEmbedding==1.0.1 sentence-transformers==2.2.2 torch pdfplumber faiss-cpu


In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
#from dotenv import load_dotenv
from langchain_experimental.text_splitter import SemanticChunker
import os
from langchain.embeddings import OllamaEmbeddings
import pdfplumber
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR


c:\Users\L01257675\Documents\Agosto-Diciembre2021-DELL\Ago-Diciembre2024\CADI2025\CADI-LLMs\.venv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


### Text embeddings 



In [3]:
#embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl')
embeddings = OllamaEmbeddings(
    model="llama3",
)


C:\Users\L01257675\AppData\Local\Temp\ipykernel_4920\2565223353.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


In [7]:
input_text = "happy"
vector = embeddings.embed_query(input_text)
print(vector)

[1.5488290786743164, -2.486738443374634, 2.9020471572875977, -0.8607193231582642, 0.20720511674880981, -0.5570003986358643, 1.5187187194824219, -0.3943015933036804, -0.6298984885215759, 0.04212304949760437, 0.23926715552806854, -0.4910275936126709, -0.06583049148321152, 0.20964907109737396, -0.07739923894405365, 5.094100475311279, -2.2450318336486816, 0.09156560152769089, -5.268024444580078, 0.7741804718971252, 1.560036063194275, 0.11157594621181488, 0.6459203958511353, 2.0061752796173096, -0.653450608253479, 0.5794245004653931, 4.289140224456787, 4.560290336608887, 2.2872676849365234, -2.1636197566986084, -1.3417202234268188, -0.29342299699783325, 0.45010271668434143, 0.45850539207458496, 1.00813889503479, -0.7066512107849121, 1.7438452243804932, 4.602950572967529, -0.1858588606119156, 0.8784586787223816, 3.422483444213867, -1.809801697731018, 3.019196033477783, -1.3088818788528442, 4.113683223724365, -1.3007351160049438, 1.030662178993225, 0.3691585659980774, 3.5228466987609863, -5.2

In [6]:
input_text = "Today I am happy"
vector = embeddings.embed_query(input_text)
print(vector)

[0.4625481963157654, 0.23358654975891113, 0.5062917470932007, 1.156820297241211, -2.736051321029663, -0.5968900918960571, -2.1014177799224854, -0.3933849334716797, -5.008724689483643, -0.7276281118392944, 0.6669514179229736, 0.4410373270511627, -0.5918744802474976, 0.6796049475669861, 2.1950602531433105, 1.7273296117782593, 0.0511428564786911, -1.4999637603759766, -2.302208423614502, 1.1743741035461426, -0.9403576850891113, -1.1822816133499146, 1.37644624710083, 2.097660541534424, -1.1392639875411987, 0.00959323812276125, 2.6142568588256836, 7.437289237976074, 5.2261271476745605, 0.03667314350605011, -2.6336474418640137, 3.558255434036255, 0.9861551523208618, -0.1520906388759613, 4.602242946624756, -1.6919541358947754, -2.358233690261841, 2.7554569244384766, 4.339276313781738, 0.8257996439933777, 2.2565042972564697, -0.8401551842689514, 1.0600061416625977, 0.015009413473308086, 2.6547064781188965, 3.661463499069214, 1.0277273654937744, 1.255636215209961, 2.460361957550049, -3.016471385

### Vectorstore

Los embeddings se guardan en databases especiales (aquí se usa FAISS) que permiten almacenar y extraer de manera eficiente la información de los embeddings, comparando  el significado semántico de los queries con el significado semántico de las palabras almacenadas en los embeddings.

### Carga y lee el documento pdf

In [8]:
pdf_path="PeterPan.pdf"

In [9]:
text =''
with open(pdf_path,'rb') as file:
    pdf_reader = PdfReader(file)
    for page in pdf_reader.pages:
        text += page.extract_text()


In [10]:
print(text)

Hace tiempo, allá por 1880, vivía en la ciudad de Londres la familia Darling. Estaba 
formada por el señor y la señora Darling y sus hijos: Wendy, Michael y John. Sin 
olvidarnos de Nana, por supuesto, el perro niñera.  
 
Vivían felices y tranquilos hasta que Peter Pan llegó a sus vidas. Todo comenzó la 
noche en que Nana tenía el día libre y la señora Darling se quedó a cargo de sus hijos. 
Cuando todos, incluida ella, estuvieron dormidos el muchacho entró por la ventana. 
Pero entonces ella se despertó  y se asustó tanto al verle que lanzó un fuerte grito. 
Entonces apareció Nana, que cerró la ventana para evitar que saliera y acabó 
atrapando su sombra. Y así fue como la sombra de Peter Pan acabó en un cajón de la 
casa de los Darling.  
 
Una noche el señor y la señora Darling salieron a cenar a casa de los vecinos del 
número 27. Los niños se quedaron en casa al cuidado de Nana y no tardaron en 
quedarse todos dormidos.  
Pero cuando la casa estaba en silencio, entró una diminuta

### Divide el texto 

In [11]:
#text_splitter=SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

In [12]:
docs = text_splitter.split_text(text)
    


In [13]:
type(docs)

list

In [14]:
len(docs)

12

In [15]:
docs[0]

'Hace tiempo, allá por 1880, vivía en la ciudad de Londres la familia Darling. Estaba \nformada por el señor y la señora Darling y sus hijos: Wendy, Michael y John. Sin \nolvidarnos de Nana, por supuesto, el perro niñera.  \n \nVivían felices y tranquilos hasta que Peter Pan llegó a sus vidas. Todo comenzó la \nnoche en que Nana tenía el día libre y la señora Darling se quedó a cargo de sus hijos. \nCuando todos, incluida ella, estuvieron dormidos el muchacho entró por la ventana. \nPero entonces ella se despertó  y se asustó tanto al verle que lanzó un fuerte grito. \nEntonces apareció Nana, que cerró la ventana para evitar que saliera y acabó \natrapando su sombra. Y así fue como la sombra de Peter Pan acabó en un cajón de la \ncasa de los Darling.  \n \nUna noche el señor y la señora Darling salieron a cenar a casa de los vecinos del \nnúmero 27. Los niños se quedaron en casa al cuidado de Nana y no tardaron en \nquedarse todos dormidos.'

In [16]:
docs[1]

'número 27. Los niños se quedaron en casa al cuidado de Nana y no tardaron en \nquedarse todos dormidos.  \nPero cuando la casa estaba en silencio, entró una diminuta hada revoloteando a gran \nvelocidad y tras ella, Peter Pan, dispuesto a recuperar su sombra. La encontró en el \ncajón en el que la había guardado Nana pero se entristeció mucho cuando comprobó \nque la sombra no le seguía. Probó a pegársela con jabón pero no dio resultado y \ndesesperado se se ntó en el suelo a llorar.  \n \n- ¿Quién está llorando? - preguntó Wendy, a quien despertaron los sollozos.  \n- Soy yo - contestó Peter  \n- ¿Cómo te llamas? - preguntó la niña, aunque ella estaba casi segura de saber quien \nera \n- Peter Pan  \n- ¿Y qué te pasa Peter?  \n- Que no consigo que mi sombra se me quede pegada  \n- Tranquilo. Creo que podré cosértela  \n \nWendy ayudó a Peter y mientras los dos niños comenzaron a hacerse amigos.  \n \n- Yo vivo en el País de Nunca Jamás. Es maravilloso, allí eres siempre un niño y no'

### Crea el vectorstore

In [17]:
vectorstore = FAISS.from_texts(docs, embeddings)


### Guardar el vectorstore


In [18]:
vectorstore.save_local("faiss_index")

### Cargar el vectorstore

In [ ]:
vectorstore = FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)
